# 20春_Web数据挖掘
*  本周主要内容：总观使用
    * requests-html 丶
    * pd.read_html 丶及
    * **requests + lxml**
*  20春_Web数据挖掘_week01
*  电子讲义设计者：廖汉腾, 许智超

-----
week01_prj02_例 使用  requests + lxml

* HTTP状态狗 取/存图片及img alt文字
* 豆瓣电影排行榜
* 再试试运气 
   * pandas [pandas.read_html()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_html.html)取出数据说用了lxml


# requests + lxml

requests-html是整合requests、lxml等模塊的新模塊，所以requests 及 lxml還是可以分開始用的，原設計就是:

* HTTP 由 requests處理
* HTML 由 lxml解析

In [1]:
URL_src = { "HTTP状态狗" : "https://httpstatusdogs.com/",
           "豆瓣电影排行榜" : "https://movie.douban.com/chart"  }

## 项目2A. HTTP状态狗
### pd.read_html 试运气

In [2]:
import pandas as pd
df = pd.read_html(URL_src['HTTP状态狗'], encoding="utf8", header=0, index_col=0)

HTTPError: HTTP Error 403: Forbidden

### 😱 HTTP Error 403: Forbidden
### 改试 requests + lxml

In [3]:
import requests
r = requests.get(URL_src['HTTP状态狗'])
print (r.status_code, r.content)

200 b'<!doctype html>\n<html lang="en">\n<head>\n<meta charset="utf-8" />\n<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" />\n<title>HTTP Status Dogs</title>\n<meta name="rating" content="general" />\n<meta name="subject" content="Hypertext Transfer Protocol Response status codes. And dogs." />\n<meta name="author" content="Mike Lee" />\n<meta name="publisher" content="HTTP Status Dogs" />\n<meta name="copyright" content="Mike Lee" />\n<meta name="host" content="httpstatusdogs.com" />\n<meta name="description" content="HTTP Status Dogs. Hypertext Transfer Protocol Response status codes. And dogs." />\n<meta name="viewport" content="width=device-width, initial-scale=1">\n<meta property="og:type" content="website" />\n<meta property="og:title" content="HTTP Status Dogs" />\n<meta property="og:description" content="HTTP Status Dogs. Hypertext Transfer Protocol Response status codes. And dogs." />\n<meta property="og:image" content="https://httpstatusdogs.com/img/200.jpg" />

### 😉 HTTP 200  OK
![418](https://httpstatusdogs.com/img/200.jpg)

###  lxml + chrome
接下来需要选择器selector抽取数据img，借用HTML的"结构"性(chrome要从页面选到结构，你懂的)，有两种可能，必记下来
* xpath 
* CSS selector

试第一个和另一个，看看有什麽规律 (侦探活)
Chrome Console下使用Javascript
* xpath: 
```$x('放這')```

* CSS: 
```$('放這')```
    

* xpath
  * /html/body/main/div/div[2]/a/img
  * /html/body/main/div/div[23]/a/img
* CSS
  * body > main > div > div:nth-child(2) > a > img
  * body > main > div > div:nth-child(23) > a > img
  
推论: xpath [2] [23] 移掉,   CSS  (2)  (23) 移掉

* $x("/html/body/main/div/div/a/img")
* $("body > main > div > div > a > img")


In [4]:
import lxml
text_data = r.content
tree = lxml.html.fromstring(text_data)  # 当成片语/成语记下来  lxml模块下的html处理子物件模块，从字符串转成html物件tree
树 = lxml.html.fromstring(text_data)  # 用中文当变数名也行, 试给你看, 和上面的代码二选一就可以
HTTP状态狗 = 树.xpath("/html/body/main/div/div/a/img")  # 当成片语/成语记下来, html物件树，有方法xpath抽数据出来
print ( HTTP状态狗 )

[<Element img at 0x1f0bba82e58>, <Element img at 0x1f0bba82ae8>, <Element img at 0x1f0bba82958>, <Element img at 0x1f0bba82908>, <Element img at 0x1f0bba82ef8>, <Element img at 0x1f0bba82d68>, <Element img at 0x1f0bba82db8>, <Element img at 0x1f0bba82b38>, <Element img at 0x1f0bba829f8>, <Element img at 0x1f0bba82c78>, <Element img at 0x1f0bc079b88>, <Element img at 0x1f0bc0794a8>, <Element img at 0x1f0bc0798b8>, <Element img at 0x1f0bc097f98>, <Element img at 0x1f0bc097ef8>, <Element img at 0x1f0bc097ea8>, <Element img at 0x1f0bc09d138>, <Element img at 0x1f0bc09d0e8>, <Element img at 0x1f0bc0ad778>, <Element img at 0x1f0bc0ad638>, <Element img at 0x1f0bc0ad138>, <Element img at 0x1f0bc0bed68>, <Element img at 0x1f0bc0a4ae8>, <Element img at 0x1f0bc0c3278>, <Element img at 0x1f0bc0c32c8>, <Element img at 0x1f0bc0c36d8>, <Element img at 0x1f0bc0c3318>, <Element img at 0x1f0bc0c3728>, <Element img at 0x1f0bc0c3778>, <Element img at 0x1f0bc0c37c8>, <Element img at 0x1f0bc0c3818>, <Elemen

In [5]:
a = HTTP状态狗[0]  #  https://lxml.de/3.1/api/private/lxml.html.HtmlElement-class.html
a.get("alt")

'100 Continue'

In [6]:
# 用列表推導 公式 [ x做些啥 for x in 列表]
# 抽 element img 裡的 alt文字
list_img_alt = [ x.get("alt") for x in HTTP状态狗]
print (list_img_alt)

['100 Continue', '200 OK', '201 Created', '202 Accepted', '203 Non-Authoritative Information', '204 No Content', '206 Partial Content', '207 Multi-Status', '208 Already Reported', '226 IM Used', '300 Multiple Choices', '301 Moved Permanently', '302 Found', '303 See Other', '304 Not Modified', '305 Use Proxy', '306 Switch Proxy', '307 Temporary Redirect', '308 Permanent Redirect', '400 Bad Request', '401 Unauthorized', '402 Payment Required', '403 Forbidden', '404 Not Found', '405 Method Not Allowed', '406 Not Acceptable', '407 Proxy Authentication Required', '408 Request Timeout', '409 Conflict', '410 Gone', '411 Length Required', '412 Precondition Failed', '413 Request Entity Too Large', '414 Request-URI Too Long', '416 Requested Range Not Satisfiable', '417 Expectation Failed', "418 I'm a teapot", '420 Enhance Your Calm', '422 Unprocessable Entity', '423 Locked', '424 Failed Dependency', '425 Unordered Collection', '426 Upgrade Required', '429 Too Many Requests', '431 Request Header 

In [7]:
# 抽 url, src
list_img_src = [ x.get("src") for x in HTTP状态狗]
print (list_img_src)

['img/100.jpg', 'img/200.jpg', 'img/201.jpg', 'img/202.jpg', 'img/203.jpg', 'img/204.jpg', 'img/206.jpg', 'img/207.jpg', 'img/208.jpg', 'img/226.jpg', 'img/300.jpg', 'img/301.jpg', 'img/302.jpg', 'img/303.jpg', 'img/304.jpg', 'img/305.jpg', 'img/306.jpg', 'img/307.jpg', 'img/308.jpg', 'img/400.jpg', 'img/401.jpg', 'img/402.jpg', 'img/403.jpg', 'img/404.jpg', 'img/405.jpg', 'img/406.jpg', 'img/407.jpg', 'img/408.jpg', 'img/409.jpg', 'img/410.jpg', 'img/411.jpg', 'img/412.jpg', 'img/413.jpg', 'img/414.jpg', 'img/416.jpg', 'img/417.jpg', 'img/418.jpg', 'img/420.jpg', 'img/422.jpg', 'img/423.jpg', 'img/424.jpg', 'img/425.jpg', 'img/426.jpg', 'img/429.jpg', 'img/431.jpg', 'img/444.jpg', 'img/450.jpg', 'img/451.jpg', 'img/494.jpg', 'img/500.jpg', 'img/501.jpg', 'img/502.jpg', 'img/503.jpg', 'img/504.jpg', 'img/506.jpg', 'img/507.jpg', 'img/508.jpg', 'img/509.jpg', 'img/510.jpg']


In [11]:
# 是相對位置, 記得要和网站位置合起來使用
# 合起來的方法, 記得使用  urllib.parse.urljoin
# 网站位置可以用http request的反嚮 response.url, 這裡剛是叫r的變量

import urllib.parse
# urllib.parse.urljoin(url1, url2)

url_base = r.url
urllib.parse.urljoin( url_base, list_img_src[0])

'https://httpstatusdogs.com/img/100.jpg'

#### 下图片
* 备好目录 (!mkdir 目录) 不是Python代码, 是系统shell
* 用requests存raw, 而非文字
* [How to download image using requests](https://stackoverflow.com/questions/13137817/how-to-download-image-using-requests)
* 以后再细说, 这下载要花点时间

In [12]:
!mkdir img

子目录或文件 img 已经存在。


In [13]:
import requests
import shutil

for imgfilename in list_img_src:
    path = urllib.parse.urljoin( url_base, imgfilename)
    
    resp = requests.get(path, stream=True)
    if r.status_code == 200:
        with open(imgfilename, mode="wb") as f:    # mode = write binary
            resp.raw.decode_content = True
            shutil.copyfileobj(resp.raw, f) 
    del resp
            
# 有些下载okay有些不行, 见img

## 项目2B. 豆瓣电影排行榜
### pd.read_html 试运气

In [14]:
import pandas as pd
df = pd.read_html(URL_src['豆瓣电影排行榜'], encoding="utf8", header=0, index_col=0)

HTTPError: HTTP Error 418: 

In [15]:
import requests
r = requests.get(URL_src['豆瓣电影排行榜'])
print (r.status_code, r.content)

418 b''


### 😱 HTTP 418 倒底是啥?  I am a teapot? 
![418](https://httpstatusdogs.com/img/418.jpg)

### 所以要假装自己是浏览器
* HTTP headers: Host, User-Agent (简称UA)

### 改试 requests + pd.read_html

In [16]:
import requests
from urllib import parse
_headers_ = {
        "Accept": "text/plain, */*; q=0.01", 
        "Connection": "keep-alive",
        "Host" : "movie.douban.com", 
        "User-Agent" : "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3250.0 Iron Safari/537.36",
}
s = requests.Session()
u = URL_src['豆瓣电影排行榜']
r = s.get(u, headers=_headers_)

In [17]:
print (r.status_code, r.content)

200 b'<!DOCTYPE html>\n<html lang="zh-cmn-Hans" class="ua-windows ua-webkit">\n<head>\n    <meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n    <meta name="renderer" content="webkit">\n    <meta name="referrer" content="always">\n    <meta name="google-site-verification" content="ok0wCgT20tBBgo9_zat2iAcimtN4Ftf5ccsh092Xeyw" />\n    <title>\n\xe8\xb1\x86\xe7\x93\xa3\xe7\x94\xb5\xe5\xbd\xb1\xe6\x8e\x92\xe8\xa1\x8c\xe6\xa6\x9c\n</title>\n    \n    <meta name="baidu-site-verification" content="cZdR4xxR7RxmM4zE" />\n    <meta http-equiv="Pragma" content="no-cache">\n    <meta http-equiv="Expires" content="Sun, 6 Mar 2005 01:00:00 GMT">\n    \n    <meta name="keywords" content="\xe7\x94\xb5\xe5\xbd\xb1\xe6\x8e\x92\xe8\xa1\x8c\xe6\xa6\x9c\xe3\x80\x81\xe6\x96\xb0\xe7\x89\x87\xe6\x8e\x92\xe8\xa1\x8c\xe6\xa6\x9c\xe3\x80\x81\xe8\xb1\x86\xe7\x93\xa3\xe7\x94\xb5\xe5\xbd\xb1250"/>\n    <meta name="description" content="\xe8\xb1\x86\xe7\x93\xa3\xe7\x94\xb5\xe5\xbd\xb1\xe6\x8e\x92\

In [18]:
df = pd.read_html(r.content, encoding="utf8", header=0, index_col=0)
df

[Empty DataFrame
 Columns: []
 Index: [], Empty DataFrame
 Columns: [1917  / 1917：逆战救兵(港) / 一九一七  2019-12-25(美国) / 2020-01-10(英国) / 2020(中国大陆) / 乔治·麦凯 / 迪恩·查尔斯·查普曼 / 科林·费尔斯 / 本尼迪克特·康伯巴奇 / 马克·斯特朗 / 理查德·麦登 / 安德鲁·斯科特 / 丹尼尔·梅斯 / 阿德里安·斯卡伯勒 / 约翰·霍林沃思 / 克里斯·瓦利...  8.5  (152102人评价)]
 Index: [], Empty DataFrame
 Columns: [小妇人  / 她们(台)  2019-12-25(美国) / 2020(中国大陆) / 西尔莎·罗南 / 艾玛·沃森 / 弗洛伦斯·皮尤 / 伊莱扎·斯坎伦 / 劳拉·邓恩 / 蒂莫西·柴勒梅德 / 梅丽尔·斯特里普 / 鲍勃·奥登科克 / 詹姆斯·诺顿 / 路易·加瑞尔 / 克里斯·库珀 / 崔西·莱茨 / 艾比·奎因...  8.1  (118362人评价)]
 Index: [], Empty DataFrame
 Columns: [寄生虫  / 寄生上流(台) / 上流寄生族(港)  2019-05-21(戛纳电影节) / 2019-05-30(韩国) / 宋康昊 / 李善均 / 赵汝贞 / 崔宇植 / 朴素丹 / 李静恩 / 张慧珍 / 郑知晓 / 朴明勋 / 朴叙俊 / 朴根祿 / 郑贤俊 / 朴孝新 / 安德烈亚斯·弗龙克 / 郑益汉 / 李东勇 / 李柱亨 / 林艺恩 / 韩国 / 奉俊昊...  8.7  (894668人评价)]
 Index: [], Empty DataFrame
 Columns: [南山的部长们  / The Man Standing Next  2020-01-22(韩国) / 李秉宪 / 李星民 / 郭度沅 / 李熙俊 / 金素真 / 徐贤宇 / 朴智一 / 李泰亨 / 金昇勋 / 金明善 / 朴成根 / 郑钟宇 / 金民尚 / 周锡泰 / 成民秀 / 赵惠珠 / 孙炳旭 / 朴炫宇 / 严志满 / 柳正浩 / 金弘波 / 李东国 / 韩国 / 禹民镐...  8.2  (49775人评价)]
 I

### 改试 requests + lxml
该您了? 本周小作业

1. 极易: prj00_例 liepin工作 找另外的工作 or prj00_例 图片新闻_Bing 找另外的图片
2. 易: prj00_例 liepin工作 找另外的工作用不同的限定不同城市
3. 中: prj00_例 liepin工作 找另外的工作 且 抽取（extract）出 薪水丶城市等信息
3. 中: prj00_例 图片新闻_Bing 找另外的图片 找另外的工作 且 抽取出 图片丶图说等信息
4. 难:  prj00_例 liepin工作 找另外的工作 且 抽取出 薪水丶城市等信息
5. 难: prj00_例 图片新闻_Bing 你如何按栏目(i.e. 今日热图, 明星, 美女) 抽取出 图片丶图说等信息?
6. 难: 豆瓣电影排行榜 你如何 ETL ( 抽取（extract）丶转换（transform）丶加载（load）至目的端) ?

每个人需提交一个Github Jekyll 的本周小作业笔记的URL，建议是
* 至少一个ipynb文档 或 
* 一个markdown文档加一个ipynb文档(提交的入口点第一部分内容需要有另一个相关文档的链结及简要说明)




---
结束
